In [0]:
# Validando a SparkSession
spark

In [0]:
# Mostrando os pontos de montagem no cluster Databricks
display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/datalakeea44c854c651c494/landing-zone,wasbs://landing-zone@datalakeea44c854c651c494.blob.core.windows.net,
/Volume,DbfsReserved,
/volumes,DbfsReserved,
/,DatabricksRoot,
/volume,DbfsReserved,


In [0]:
# Desmontando os pontos de montagem caso necessário
 
dbutils.fs.unmount('/mnt/datalakeea44c854c651c494/lading-zone')
#dbutils.fs.unmount('/mnt/datalakeea44c854c651c494/bronze')
#dbutils.fs.unmount('/mnt/datalakeea44c854c651c494/silver')
#dbutils.fs.unmount('/mnt/datalakeea44c854c651c494/gold')

In [0]:
# Definindo uma função para montar um ADLS com um ponto de montagem com ADLS SAS

storageAccountName = "datalakeea44c854c651c494"
storageAccountAccessKey = ""
sasToken = "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-06-20T02:32:31Z&st=2025-06-19T18:32:31Z&spr=https&sig=CGev8c8IkJuIFAGEeFukBHu%2B66q2MxFaeswMz0QNSVo%3D"

def mount_adls(blobContainerName):
    try:
      dbutils.fs.mount(
        source = "wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
        mount_point = f"/mnt/{storageAccountName}/{blobContainerName}",
        #extra_configs = {'fs.azure.account.key.' + storageAccountName + '.blob.core.windows.net': storageAccountAccessKey}
        extra_configs = {'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
      )
      print(f"Montagem bem-sucedida: {blobContainerName}")
    except Exception as e:
      print(f"Falha na montagem de {blobContainerName}:", e)

In [0]:
# Montando todos os containers

mount_adls('landing-zone')
mount_adls('bronze')
mount_adls('silver')
mount_adls('gold')

Falha na montagem de landing-zone: An error occurred while calling o433.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/datalakeea44c854c651c494/landing-zone; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/datalakeea44c854c651c494/landing-zone
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:152)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:74)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:1205)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.$anonfun$mount$1(DBUtilsCore.scala:1231)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:528)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:633)
	at com.databricks.logging.UsageLogging.$anonfun$recordO

In [0]:
# Mostrando os pontos de montagem no cluster Databricks

display(dbutils.fs.mounts())

mountPoint,source,encryptionType
/databricks-datasets,databricks-datasets,
/Volumes,UnityCatalogVolumes,
/databricks/mlflow-tracking,databricks/mlflow-tracking,
/databricks-results,databricks-results,
/mnt/datalakeea44c854c651c494/gold,wasbs://gold@datalakeea44c854c651c494.blob.core.windows.net,
/databricks/mlflow-registry,databricks/mlflow-registry,
/mnt/datalakeea44c854c651c494/bronze,wasbs://bronze@datalakeea44c854c651c494.blob.core.windows.net,
/mnt/datalakeea44c854c651c494/landing-zone,wasbs://landing-zone@datalakeea44c854c651c494.blob.core.windows.net,
/Volume,DbfsReserved,
/mnt/datalakeea44c854c651c494/silver,wasbs://silver@datalakeea44c854c651c494.blob.core.windows.net,


In [0]:
# Mostrando todos os arquivos da camada landing-zone
 
display(dbutils.fs.ls(f"/mnt/{storageAccountName}/landing-zone"))

path,name,size,modificationTime
dbfs:/mnt/datalakeea44c854c651c494/landing-zone/circuits.csv,circuits.csv,10104,1750347523000
dbfs:/mnt/datalakeea44c854c651c494/landing-zone/constructor_results.csv,constructor_results.csv,219365,1750347523000
dbfs:/mnt/datalakeea44c854c651c494/landing-zone/constructor_standings.csv,constructor_standings.csv,317206,1750347523000
dbfs:/mnt/datalakeea44c854c651c494/landing-zone/constructors.csv,constructors.csv,17478,1750347523000
dbfs:/mnt/datalakeea44c854c651c494/landing-zone/driver_standings.csv,driver_standings.csv,883771,1750347523000
dbfs:/mnt/datalakeea44c854c651c494/landing-zone/drivers.csv,drivers.csv,94367,1750347523000
dbfs:/mnt/datalakeea44c854c651c494/landing-zone/lap_times.csv,lap_times.csv,17622395,1750347524000
dbfs:/mnt/datalakeea44c854c651c494/landing-zone/pit_stops.csv,pit_stops.csv,443719,1750347523000
dbfs:/mnt/datalakeea44c854c651c494/landing-zone/qualifying.csv,qualifying.csv,465231,1750347523000
dbfs:/mnt/datalakeea44c854c651c494/landing-zone/races.csv,races.csv,187042,1750347523000


In [0]:
# Gerando um dataframe para cada arquivo a partir dos arquivos CSV e gravado no container landing-zone do Azure Data Lake Storage

df_circuits = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/circuits.csv")
df_constructor_results   = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/constructor_results.csv")
df_constructor_standings = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/constructor_standings.csv")
df_constructors     = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/constructors.csv")
df_driver_standings = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/driver_standings.csv")
df_drivers    = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/drivers.csv")
df_lap_times  = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/lap_times.csv")
df_pit_stops  = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/pit_stops.csv")
df_qualifying = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/qualifying.csv")
df_races   = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/races.csv")
df_results = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/results.csv")
df_seasons = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/seasons.csv")
df_sprint_results = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/sprint_results.csv")
df_status  = spark.read.option("infeschema", "true").option("header", "true").csv(f"/mnt/{storageAccountName}/landing-zone/status.csv")

In [0]:
# Adicionando metadados de data e hora de processamento e nome do arquivo de origem

from pyspark.sql.functions import current_timestamp, lit

df_circuits = df_circuits.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("circuits.csv"))
df_constructor_results   = df_constructor_results.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("constructor_results.csv"))
df_constructor_standings = df_constructor_standings.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("constructor_standings.csv"))
df_constructors  = df_constructors.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("constructors.csv"))
df_driver_standings = df_driver_standings.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("driver_standings.csv"))
df_drivers       = df_drivers.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("drivers.csv"))
df_lap_times     = df_lap_times.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("lap_times.csv"))
df_pit_stops     = df_pit_stops.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("pit_stops.csv"))
df_qualifying    = df_qualifying.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("qualifying.csv"))
df_races         = df_races.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("races.csv"))
df_results       = df_results.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("results.csv"))
df_seasons       = df_seasons.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("seasons.csv"))
df_sprint_results = df_sprint_results.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("sprint_results.csv"))
df_status        = df_status.withColumn("data_hora_bronze", current_timestamp()).withColumn("nome_arquivo", lit("status.csv"))

In [0]:
# Salvando os dataframes em delta lake (formato de arquivo) no Data Lake (repositorio cloud)

df_circuits.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/circuits")
df_constructor_results.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/constructor_results")
df_constructor_standings.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/constructor_standings")
df_constructors.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/constructors")
df_driver_standings.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/driver_standings")
df_drivers.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/drivers")
df_lap_times.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/lap_times")
df_pit_stops.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/pit_stops")
df_qualifying.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/qualifying")
df_races.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/races")
df_results.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/results")
df_seasons.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/seasons")
df_sprint_results.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/sprint_results")
df_status.write.format('delta').save(f"/mnt/{storageAccountName}/bronze/status")

In [0]:
# Verificando os dados gravados em delta na camada BRONZE

display(dbutils.fs.ls(f"/mnt/{storageAccountName}/bronze/"))

path,name,size,modificationTime
dbfs:/mnt/datalakeea44c854c651c494/bronze/circuits/,circuits/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/constructor_results/,constructor_results/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/constructor_standings/,constructor_standings/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/constructors/,constructors/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/driver_standings/,driver_standings/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/drivers/,drivers/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/lap_times/,lap_times/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/pit_stops/,pit_stops/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/qualifying/,qualifying/,0,0
dbfs:/mnt/datalakeea44c854c651c494/bronze/races/,races/,0,0


In [0]:
# Consultando um exemplo de um delta lake para validar a existência dos dados e das colunas do metadados na camada BRONZE

spark.read.format('delta').load(f'/mnt/{storageAccountName}/bronze/drivers').limit(10).display()

driverId,driverRef,number,code,forename,surname,dob,nationality,url,data_hora_bronze,nome_arquivo
1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton,2025-06-19T21:54:03.058Z,drivers.csv
2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld,2025-06-19T21:54:03.058Z,drivers.csv
3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg,2025-06-19T21:54:03.058Z,drivers.csv
4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso,2025-06-19T21:54:03.058Z,drivers.csv
5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen,2025-06-19T21:54:03.058Z,drivers.csv
6,nakajima,\N,NAK,Kazuki,Nakajima,1985-01-11,Japanese,http://en.wikipedia.org/wiki/Kazuki_Nakajima,2025-06-19T21:54:03.058Z,drivers.csv
7,bourdais,\N,BOU,Sébastien,Bourdais,1979-02-28,French,http://en.wikipedia.org/wiki/S%C3%A9bastien_Bourdais,2025-06-19T21:54:03.058Z,drivers.csv
8,raikkonen,7,RAI,Kimi,Räikkönen,1979-10-17,Finnish,http://en.wikipedia.org/wiki/Kimi_R%C3%A4ikk%C3%B6nen,2025-06-19T21:54:03.058Z,drivers.csv
9,kubica,88,KUB,Robert,Kubica,1984-12-07,Polish,http://en.wikipedia.org/wiki/Robert_Kubica,2025-06-19T21:54:03.058Z,drivers.csv
10,glock,\N,GLO,Timo,Glock,1982-03-18,German,http://en.wikipedia.org/wiki/Timo_Glock,2025-06-19T21:54:03.058Z,drivers.csv
